# 🔄 Notebook: 01 ETL Silver Layer

This notebook handles the **Silver Layer (Data Transformation & Processing)** in the Medallion Architecture of the AI-powered claims pipeline. It focuses on **audio conversion**, **metadata extraction**, and **speech-to-text transcription** using the [OpenAI Whisper model](https://openai.com/index/whisper/).

---

## 🧱 Purpose

To convert raw audio files into a consistent format (MP3), calculate metadata (duration), and transcribe the content into structured text to support downstream AI analytics.



In [0]:
%pip install pydub mutagen openai-whisper numpy>=1.24
dbutils.library.restartPython()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/800.5 kB ? eta -:--:--
     ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/800.5 kB 5.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━ 501.8/800.5 kB 7.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 798.7/800.5 kB 8.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/194.4 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/63.0 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.

In [0]:
%run "./resources/init" 

DataFrame[]

In [0]:
from pydub import AudioSegment
import os

file_reference_df = spark.table(f"{CATALOG}.{SCHEMA}.recordings_file_reference_bronze")

mp3_path = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/mp3_audio_recordings/"
if not dbutils.fs.mkdirs(mp3_path):
    dbutils.fs.mkdirs(mp3_path)

# Convert each file to mp3 and save to the new volume
for row in file_reference_df.collect():
    file_path = row['file_path']
    audio = AudioSegment.from_file(file_path)
    new_file_path = os.path.join(mp3_path, os.path.basename(file_path).replace(os.path.splitext(file_path)[1], ".mp3"))
    audio.export(new_file_path, format="mp3")

In [0]:
from mutagen.mp3 import MP3
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

mp3_file_reference_df = spark.createDataFrame(
    dbutils.fs.ls(f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/mp3_audio_recordings")
).withColumn("file_path", F.expr("substring(path, 6, length(path))"))

def get_audio_duration(file_path):
    audio = MP3(file_path)
    return audio.info.length

get_audio_duration_udf = F.udf(get_audio_duration, FloatType())

mp3_file_reference_df = mp3_file_reference_df.withColumn("audio_duration", F.round(get_audio_duration_udf("file_path"), 0))

display(mp3_file_reference_df)

path,name,size,modificationTime,file_path,audio_duration
dbfs:/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,724461,1743602169000,/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,91.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,865197,1743602161000,/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,108.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,nv7032f9_AGT001_2025-02-27 12_40_45.mp3,914349,1743602163000,/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,114.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/pxvlh18a_AGT001_2025-02-11 11_33_33.mp3,pxvlh18a_AGT001_2025-02-11 11_33_33.mp3,946605,1743602165000,/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/pxvlh18a_AGT001_2025-02-11 11_33_33.mp3,118.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/ulnocrnh_AGT005_2025-02-04 05_42_51.mp3,ulnocrnh_AGT005_2025-02-04 05_42_51.mp3,956013,1743602167000,/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/ulnocrnh_AGT005_2025-02-04 05_42_51.mp3,119.0


In [0]:
import whisper

# Load Whisper model (choose "small" for CPU, "medium" or "large" for GPU)
model = whisper.load_model("small")
print("Model loaded successfully!")

Error in cpuinfo: prctl(PR_SVE_GET_VL) failed
100%|███████████████████████████████████████| 461M/461M [00:08<00:00, 56.8MiB/s]
/local_disk0/.ephemeral_nfs/envs/pythonEnv-16e76eef-f2e9-4025-9d20-1e88c083243e/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where 

Model loaded successfully!


In [0]:
def transcribe_audio(file_path: str, model: whisper.Whisper) -> str:
    """
    Transcribe audio using Whisper model.

    Args:
        file_path (str): Path to the audio file.
        model (whisper.Whisper): Whisper model instance.

    Returns:
        str: Transcribed text from the audio file.
    """
    result = model.transcribe(file_path)
    return result["text"]

In [0]:
from pyspark.sql.functions import col, split, regexp_replace, to_timestamp, concat_ws
from pyspark.sql.types import StringType

# Collect the file paths to the driver
file_paths = mp3_file_reference_df.select("file_path").rdd.flatMap(lambda x: x).collect()

# Transcribe the audio files outside of Spark
transcriptions = [transcribe_audio(file_path, model) for file_path in file_paths]

# Create a DataFrame with the transcriptions
transcriptions_df = spark.createDataFrame(zip(file_paths, transcriptions), ["file_path", "transcription"])

# Join the transcriptions back to the original DataFrame
result_df = mp3_file_reference_df.join(transcriptions_df, on="file_path", how="inner") \
                                 .select("path", "modificationTime", "file_path", "transcription", "audio_duration")

result_df = result_df.withColumn("file_name", split(col("file_path"), "/").getItem(6)) \
    .withColumn("file_name", regexp_replace(col("file_name"), ".mp3", "")) \
    .withColumn("call_id", split(col("file_name"), "_").getItem(0)) \
    .withColumn("agent_id", split(col("file_name"), "_").getItem(1)) \
    .withColumn("call_datetime", 
        to_timestamp(
            concat_ws(":", split(col("file_name"), "_").getItem(2), 
            split(col("file_name"), "_").getItem(3), 
            split(col("file_name"), "_").getItem(4)))
    )

display(result_df)

path,modificationTime,file_path,transcription,audio_duration
dbfs:/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,1743602169000,/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,"Hello, you've reached out Vital Guard, this is John speaking. How can I assist you today? Hi, this is Sophia Wilson. I was hoping to change my payment method. Of course, Sophia. For security, can I confirm your date of birth and policy number, please? Sure. My date of birth is 15th August 1995 and my policy number is VG924695. Thank you for confirming. What is your current payment method and how would you like to change it? I currently pay by direct debit, but I would like to switch to paying by credit card. Hmm, I can assist you with that. Please note that changing your payment method may affect your policy terms. Would you like me to explain the details? Yes, please do. Let me check that for you. Okay, I've updated your payment method to credit card. Your new payment schedule will be sent to you via email. Also, I'd like to take this opportunity to explain your coverage details. That sounds great, thank you. You're welcome. Just to summarize, I have changed your payment method to credit card and explained your coverage details. If you need further assistance, don't hesitate to reach out and have a wonderful day. Thanks, you too.",91.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,1743602161000,/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,"Hello, this is Vital Guard, you are speaking to John, how can I help you today? Hi, this is Emma Johnson, I'm hoping to get some advice on my coverage options. Of course Emma, for security, can I confirm your date of birth and policy number please? Sure, my DOB is the 22nd of July 1990 and my policy number is VG 434271. Thank you for confirming. What kind of coverage details are you looking for? I want to know if my policy covers physiotherapy sessions. Okay, let me check that for you. Yes, your current plan includes up to 10 physiotherapy sessions per year, but they need to be prescribed by a GP. That's great to know. Also, if I need to make a complaint about a billing issue, how do I do that? I understand. I will provide you with a complaint reference number and you can submit your complaint through our online portal or over the phone. Here is your reference number, VG-CMP-4823. Thanks, that helps. Of course, you're very welcome. Is there anything else I can help you with today? No, that's all for now. Great. Just to summarize, Emma, I have confirmed your policy covers physiotherapy with a GP referral and I have provided a complaint reference number for your billing issue. If you do need further assistance, don't hesitate to reach out. Have a wonderful day. Thanks, you too.",108.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,1743602163000,/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,"Hello, thank you for calling Vital Guard. My name is Michelle. How can I assist you today? Hey, this is Liam. Liam Brown. And I was hoping to get a duplicate policy document. Of course, Liam. For security, can I confirm your date of birth and policy number? Sure. It's 10th of May, 1978, and my policy number is VG82910. Thank you for confirming. I understand you're requesting a duplicate policy document. Can you please tell me what happened to the original document? I lost it during the move, and now I need it for my records. I apologize for the inconvenience, Liam. I'm going to go ahead and send you a duplicate policy document via email. You sh

In [0]:
# existing_transcriptions_df = spark.table(f"{CATALOG}.{SCHEMA}.simulated_transcriptions")
# combined_transcriptions_df = existing_transcriptions_df.unionByName(result_df)

# combined_transcriptions_df = combined_transcriptions_df.withColumn("file_name", split(col("file_path"), "/").getItem(6)) \
#     .withColumn("file_name", regexp_replace(col("file_name"), ".mp3", "")) \
#     .withColumn("call_id", split(col("file_name"), "_").getItem(0)) \
#     .withColumn("agent_id", split(col("file_name"), "_").getItem(1)) \
#     .withColumn("call_datetime", 
#         to_timestamp(
#             concat_ws(":", split(col("file_name"), "_").getItem(2), 
#             split(col("file_name"), "_").getItem(3), 
#             split(col("file_name"), "_").getItem(4))))
    

# display(combined_transcriptions_df)

# combined_transcriptions_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{CATALOG}.{SCHEMA}.transcriptions_silver")

path,modificationTime,file_path,transcription,audio_duration,file_name,call_id,agent_id,call_datetime
dbfs:/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/20156e21_AGT005_2025-02-23 14:36:43.mp3,1743601995,/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/20156e21_AGT005_2025-02-23 14:36:43.mp3,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Olivia Flores. I was hoping to get some information about my coverage details. Of course, Olivia. For security, can I confirm your date of birth and policy number? Sure, my DOB is 8th July 1960, and my policy number is VG382370. Thank you for confirming. What kind of coverage details are you looking for? I want to know what is included in my policy and if there are any exclusions. Let me check that for you... Your policy includes coverage for hospital stays, doctor visits, and prescriptions. However, it does not cover pre-existing conditions or cosmetic procedures. That's helpful to know. Are there any limits to the number of doctor visits I can have? Yes, your policy allows for up to 10 doctor visits per year, but you will need to get a referral from your GP for any specialist visits. Okay, that makes sense. Is there anything else I can assist you with today? No, that's all for now. Great! Just to summarize, I have explained your coverage details, including what is included and excluded, and the limits on doctor visits. If you need further assistance, don’t hesitate to reach out. Have a wonderful day! Thanks! Goodbye!",200.0,20156e21_AGT005_2025-02-23 14:36:43,20156e21,AGT005,2025-02-23T14:36:43Z
dbfs:/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/c6fc70df_AGT004_2025-02-07 08:15:43.mp3,1743601995,/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/c6fc70df_AGT004_2025-02-07 08:15:43.mp3,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Isabella Davis. I was hoping to file a new claim. Of course, Isabella. For security, can I confirm your date of birth and policy number? Sure, my DOB is 24th July 1982, and my policy number is VG813971. Thank you for confirming. Can you please provide more details about the claim you'd like to file? I recently had a medical procedure and I'd like to claim for the expenses. I understand. I'm going to guide you through the process of filing a new claim. Please allow me to check on the details of your policy... Okay, it seems like you are eligible to file a claim for the medical procedure. I will now file a new claim request on your behalf. Thank you so much! I appreciate your help. You're welcome, Isabella. Just to summarize, I have filed a new claim request for you. You will receive an email with the claim reference number and further instructions. If you have any questions or need further assistance, don't hesitate to reach out. Is there anything else I can assist you with today? No, that's all for now. Thank you again. You're welcome, Isabella. Have a great day and feel free to contact us if you need anything else.",134.0,c6fc70df_AGT004_2025-02-07 08:15:43,c6fc70df,AGT004,2025-02-07T08:15:43Z
dbfs:/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/6f7190aa_AGT005_2025-03-23 06:24:43.mp3,1743601995,/Volumes/samantha_wise/ai_claims_processing_final/audio_recordings/mp3_audio_recordings/6f7190aa_AGT005_2025-03-23 06:24:43.mp3,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Riley Perez. I was hoping to get an update on my claim status. Of course, Riley. For security, can I confirm your date of birth and policy number? Sure, my DOB is 22nd February 1999, and my policy number is VG201105. Thank you for confirming. I've located your claim. Can you please tell me a little bit more about what's going on with your claim and 

In [0]:
if spark.catalog.tableExists(f"{CATALOG}.{SCHEMA}.transcriptions_silver"):
    transcriptions_silver_df = spark.table(f"{CATALOG}.{SCHEMA}.transcriptions_silver")
    combined_transcriptions_df = transcriptions_silver_df.unionByName(result_df).dropDuplicates()
else:
    combined_transcriptions_df = result_df.dropDuplicates()

combined_transcriptions_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{CATALOG}.{SCHEMA}.transcriptions_silver")

display(combined_transcriptions_df)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3394427869729194>, line 3
      1 if spark.catalog.tableExists(f"{CATALOG}.{SCHEMA}.transcriptions_silver"):
      2     transcriptions_silver_df = spark.table(f"{CATALOG}.{SCHEMA}.transcriptions_silver")
----> 3     combined_transcriptions_df = transcriptions_silver_df.unionByName(result_df).dropDuplicates()
      4 else:
      5     combined_transcriptions_df = result_df.dropDuplicates()

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:4989, in Dat

## ✅ Output
- A clean, enriched Delta table: transcriptions_silver
- Includes transcription text, call metadata, and audio duration for each entry.